 ### <br><b> Introduction to data analysis_Part2</b>

Our objective here is to introduce you to pandasas this library has become the python workhorse for analysing any dataset that most chemists (or biologists) would naturally handle in excel.

We will do that with a data set of climate variables from GRENOBLE-ST GEOIRS AEROPORT , which are downloaded from METEO FRANCE site.
>In case your curiosity strikes you:
    <u><br>https://donneespubliques.meteofrance.fr/</u>

As getting the data through this interface is kind of clumsy, a version is available from the following file: /data/Data.xslx

This datafile contains following data:

> <u>Variables:</u>
    <br>- max/min/mean temperature
    <br>- total precipitation 
  <br><u>Temporal coverage: </u>
        <br>1968/01/01-2022/01/31

    

<br><b>Tasks:</b>

>1)
><br> Data were downloaded from:
><br> <u> http://www.temis.nl/data/gmted2010/</u>
><br> Highest resolution is used (0.0625°), file downloaded in /bettik/jomaaf/Stations/Data/</b>
><br> To read and extract the desired altitude check "elevation" personal function in /home/jomaaf/PHD/Packages/topography.py </b>


>2)
><br> Data from Olga (to be checked):
><br> Resolution (0.03°)
><br> Data can be found in: /bettik/jomaaf/Stations/altitude.txt
><br><b> Output files:</b>
      > <br>- altitude_all_0.03.nc
      > <br>- altitude_GOL_0.03.nc # bounderies GOL
      <br>saved in /bettik/jomaaf/Stations/Topography/
    

### <br><b> Introduction to data analysis_Part2</b>


In [3]:
path='/data/fjomaa/TP_2022/'

### 1)  Downloaded

In [1]:
# import libraries
import sys
sys.path.insert(1,'/home/fjomaa/PHD/Packages/') # to include my functions file in another directory
from lib_basics import *

import sys
sys.path.insert(1,'/home/fjomaa/PHD/Functions/') # to include my functions file in another directory
import zone as fu
import topography as elv

import xesmf

In [2]:
#Check domain bounderies
latlim, lonlim = fu.get_domain_GOl()
print('\nlat:',latlim.start,latlim.stop,'\nlon:',lonlim.start,lonlim.stop)


lat: 41 46 
lon: 1 11


In [9]:
## Open file using xarray
# rename coordinates and cut on your lat,lon domain 
ds = xr.open_dataset(path+'GMTED2010_15n015_00625deg.nc').drop_dims('nbounds').swap_dims(
        {'nlat': 'latitude', 'nlon': 'longitude'}).drop({'nlat', 'nlon'}).rename(
        {'latitude': 'lat', 'longitude': 'lon'}).sel(lat=latlim, lon=lonlim)
print(ds)

## Define parameters
elevation = ds.elevation             #elevation 
elevation_std = ds.elevation_stddev  #standard deviation 

<xarray.Dataset>
Dimensions:           (lat: 80, lon: 160)
Coordinates:
  * lat               (lat) float32 41.03 41.09 41.16 ... 45.84 45.91 45.97
  * lon               (lon) float32 1.031 1.094 1.156 ... 10.84 10.91 10.97
Data variables:
    elevation         (lat, lon) int16 ...
    elevation_stddev  (lat, lon) int16 ...
    elevation_max     (lat, lon) int16 ...
    elevation_min     (lat, lon) int16 ...
Attributes: (12/13)
    institution:                KNMI / TEMIS
    url:                        http://temis.nl/data/gmted2010/
    Title:                      GMTED2010 elevation data regridded at 0.062 d...
    resolution_degrees:         0.0625
    resolution_summing_factor:  15
    input_file:                 S5P_OPER_REF_DEM_15_00000000T000000_99999999T...
    ...                         ...
    input_note:                 see "Original_attributes" for a copy of the a...
    creation_date:              2019-10-15
    gmted2010_citation:         Danielson, J.J., and Gesch, D.B

### 2)  From Olga

In [7]:
altitude=path+'precipitation.txt'

## read file
df=np.loadtxt(altitude,unpack=True) #shape (lon,lat,altitude) (3,2883601)
print(df)

## Define some parameters 
#resolution 
res=(df[0][1]-df[0][0])
Station=df[0]
Date=df[1]
RR=df[2]

[[3.8384001e+07 3.8384001e+07 3.8384001e+07 ... 3.8384001e+07
  3.8384001e+07 3.8384001e+07]
 [1.9680101e+07 1.9680102e+07 1.9680103e+07 ... 2.0220129e+07
  2.0220130e+07 2.0220131e+07]
 [1.2800000e+01 6.9000000e+00 5.2000000e+00 ... 0.0000000e+00
  0.0000000e+00 1.4000000e+00]]


In [28]:
import pandas as pd

# df = pd.read_csv(path+'Data.xslx',
#                  sep=';',
#                  na_values='-',
#                 )
# df=pd.read_table(path+'Data.xslx',delimiter = ';',error_bad_lines=False)
df=pd.read_csv(path+'Data.csv',
         
                )

df_backup = df.copy()  

print(df.info())

df.head() #only prints out the first 5 lines....

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19755 entries, 0 to 19754
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Station        19755 non-null  int64  
 1   Date           19755 non-null  int64  
 2   mean_T         19752 non-null  float64
 3   min_T          19755 non-null  float64
 4   max_T          19755 non-null  float64
 5   Precipitation  19755 non-null  float64
dtypes: float64(4), int64(2)
memory usage: 926.1 KB
None


,Station,Date,mean_T,min_T,max_T,Precipitation
0,38384001,19680101,-0.7,-5.0,2.3,12.8
1,38384001,19680102,0.2,-0.8,2.0,6.9
2,38384001,19680103,-2.3,-5.2,2.8,5.2
3,38384001,19680104,1.6,-0.9,3.2,4.5
4,38384001,19680105,1.4,0.2,7.8,14.4


In [35]:
df['mean_T'][1:100]  #try others !

1      0.2
2     -2.3
3      1.6
4      1.4
5      2.4
      ... 
95    12.0
96    12.2
97     5.6
98     5.4
99     4.0
Name: mean_T, Length: 99, dtype: float64

In [6]:
# check the available bounderies of altitude 
latlim=slice(min(lat),max(lat))
lonlim=slice(min(lon),max(lon))

In [7]:
# Define latitude and longitude 
latalt=np.arange(latlim.stop,latlim.start,-res) # decreasing latitude mn 70 la 30 check df[1]
lonalt=np.arange(lonlim.start,lonlim.stop,res)

#Reshape the altitude of shape (lat*lon) into shape (lat,lon)
altitude=alt.reshape(len(latalt),len(lonalt))

In [11]:
# Save data into xarray
da= xr.DataArray(
    dims=["time"],
    coords=dict(
        time=(["time"],Date),
        RR=(["time"],RR),
        
    ),    
)
da

<xarray.DataArray (time: 19755)>
array([nan, nan, nan, ..., nan, nan, nan])
Coordinates:
  * time     (time) float64 1.968e+07 1.968e+07 ... 2.022e+07 2.022e+07
    RR       (time) float64 12.8 6.9 5.2 4.5 14.4 3.6 ... 0.0 0.0 0.0 0.0 1.4

In [9]:
# cut over GOL
da1=da.sel(lon=slice(0.964570,11.03023),lat=slice(46.04,41))


In [10]:
# save altitude 
da.to_netcdf(path+'altitude_all_0.03.nc')
da1.to_netcdf(path+'altitude_GOL_0.03.nc')
